In [ ]:
# -*- coding: utf-8 -*-
import recon
import plotting as plot
import numpy as np
import seaborn as sns
import pandas as pd
import utilities
import workflows as wf
import warnings
warnings.filterwarnings('ignore')


model = recon.Recon()
util = utilities.Utilities()
sns.set_context("talk")

### Metabolomics data

In [ ]:
met_slc25a51ko = pd.read_csv('../data/metabolomics_slc25a51ko_vs_parental.csv', 
                             sep='\t', index_col=0)
met_uc = pd.read_csv('../data/metabolomics_uc_vs_healthy.csv', sep=',')

### Predicted change

In [ ]:
res_slc25a51ko = pd.read_csv('../data/pr_slc25a51ko_wt.csv', sep='\t', index_col=0)
res_uc = pd.read_csv('../data/pr_prot_uc_vs_healthy.csv', sep='\t', index_col=0)

### RNA-Seq data

In [ ]:
rna_slc25a51ko = pd.read_csv('../data/rnaseq_slc25a51ko_vs_parental.csv', sep='\t', index_col=0)
rna_slc25a51ko = util.map_gene(df=rna_slc25a51ko, g_mapping=model.genes, mapping_column='ensembl_gene')

# Histograms

In [ ]:
# Figure 2a
plot.hist(pr=np.log2(res_slc25a51ko.mean(axis=1)), bins=11, figsize=(8, 7), xlim=(-1.0, 1.0),
          ylim=(0, 2e4), xlabel='log$_2$(predicted change)', ylabel='no. of metabolites',
          save=False, filename=None)

In [ ]:
# Figure 3a
plot.hist(pr=np.log2(res_uc.mean(axis=1)),
          figsize=(8, 7), xlim=(-1., 1.), bins=19, color='C1', ylim=(0, 2e4),
          xlabel='log$_2$(predicted change)', ylabel='no. of metabolites',
          save=False, filename=None)

# Parity plots

In [ ]:
res_HEK = util.reshape(res_slc25a51ko, include=None)
# _res = res_HEK.groupby('metabolites').mean(numeric_only=True)
_res = res_HEK[res_HEK.compartment.isin(['mitochondrial', 'nuclear', 'cytoplasm'])].groupby(
    'metabolites').mean(numeric_only=True)
df = wf.prep_data(model=model, rna=rna_slc25a51ko, mets=met_slc25a51ko, res=_res)

In [ ]:
# Figure 2b
_df = df[~df.index.isin(['D-Glucose', ])]
plot.parity(data=_df, xcolumn='Measured', ycolumn='Predicted',
            show_percentage=True, show_names=True, alpha=1.0, 
            figsize=(7, 6), save=False, filename=None, edgecolor=None,
            ylabel='log$_2$(measured change)', xlabel='log$_2$(predicted change)',
            n=[1, 1, 2, 1], fsize=9, xlim=(-0.6, 0.6), ylim=(-2.5, 2.5),
            l_p=[(0.02, 0.95), (0.65, 0.95), (0.02, 0.35), (0.65, 0.35)],
            l_n=[(0.02, 0.90), (0.65, 0.90), (0.02, 0.30), (0.65, 0.30)])

In [ ]:
# Figure 3b
df = util.reshape(res_uc).groupby('metabolites').mean(numeric_only=True).reset_index()
df = met_uc.merge(df, right_on='metabolites', left_on='vmh', how='inner')
df = df.set_index('metabolites').drop('vmh', axis=1)
df.columns = ['p', 'padj', 'Measured', 'Predicted']
df['hue'] = 'C1'
df.loc[(df.p > 0.05), 'hue'] = 'grey'
df.loc[(df.Predicted < 1.01) & (df.Predicted > 0.99), 'hue'] = 'grey'
df.loc[(df.Measured < 1.2) & (df.Measured > 0.8), 'hue'] = 'grey'

In [ ]:
plot.parity(data=df.loc[df.hue == 'C1'], xcolumn='Measured', ycolumn='Predicted',
            show_percentage=True, show_names=True, alpha=1.0,
            xlim=(-0.25, 0.25), ylim=(-2.5, 2.5), edgecolor=None,
            ylabel='log$_2$(measured change)', xlabel='log$_2$(predicted change)',
            n=[1, 1, 2, 1], figsize=(10, 9), color='C1',
            l_p=[(0.1, 0.95), (0.55, 0.95), (0.1, 0.35), (0.65, 0.35)],
            l_n=[(0.1, 0.90), (0.65, 0.95), (0.1, 0.30), (0.65, 0.30)],
            fsize=10, save=False, filename=None)

In [ ]:
# Figure S1a
_df = df[~df.index.isin(['D-Glucose', ])]
plot.parity_with_errorbars(df=_df, xcolumn='Predicted', ycolumn='Measured',
                        xerr_column='Predicted std_dev', yerr_column='Measured std_dev',
                        figsize=(8,7), xlim=(-0.5, 0.5), ylim=(-3.5, 3.5),
                        show_metabolite_names=True, show_percentage=True,
                        l_p=[(0.05, 0.95), (0.6, 0.95), (0.05, 0.35), (0.6, 0.35)],
                        l_n=[(0.15, 0.90), (0.6, 0.90), (0.05, 0.30), (0.6, 0.30)],
                        save=False, filename=None,
                        n_n=[1, 1, 2, 1])